<a href="https://colab.research.google.com/github/MMoronto/peutingar-labs/blob/master/Forecasting_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import itertools
import warnings

import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
plt.style.use('seaborn-white')
%matplotlib inline

from scipy import stats
import statsmodels.api as sm

from scipy.stats import anderson
from statsmodels.tools.eval_measures import rmse
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import month_plot, seasonal_plot, plot_acf, plot_pacf, quarter_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
from statsmodels.stats.diagnostic import acorr_ljungbox as ljung
#from nimbusml.timeseries import SsaForecaster
from statsmodels.tsa.statespace.tools import diff as diff

import statsmodels.formula.api as smf
from statsmodels.tsa.statespace.tools import diff as diff
# import pmdarima as pm
# from pmdarima import ARIMA, auto_arima

from scipy import signal
from scipy.stats import shapiro
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler

#library to use R in Python 
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
 


import warnings
warnings.filterwarnings("ignore")
np.random.seed(786)

# import rdatasets

from tqdm import tqdm

<ipython-input-1-b246338cd54b>:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [2]:
#Printing library versions

print('Pandas:', pd.__version__)
print('Statsmodels:', sm.__version__)
# print('Scipy:', scipy.__version__)
print('Rpy2:', rpy2.__version__)

Pandas: 1.5.3
Statsmodels: 0.13.5
Rpy2: 3.5.5


In [3]:
plt.rc('axes', titlesize='medium')
plt.rc('axes', titlelocation='left')
plt.rc('axes.spines', right=False)
plt.rc('axes.spines', top=False)
sizets = (8,4.5)
plt.rc('figure', figsize=sizets)


def summarize(gb, f):
    """Summarize grouped things."""
    return gb.apply(lambda x: pd.Series(f(x)))


def compute(df, f):
    """Compute new (or replacement) columns."""
    newdf = pd.DataFrame(f(df), index=df.index)
    dropcols = [col for col in newdf.columns if col in df.columns]
    if dropcols:
        df = df.drop(columns=dropcols)
    return df.join(newdf)


def set_freq(df, freq=None):
    """Set frequency of DateTimeIndex."""
    if freq is None:
        freq = pd.infer_freq(df.index)
    return df.asfreq(freq)


def extend_timeseries(df, tmax=None, tmin=None, dt=None):
    """Extend timeseries data to later or earlier times."""
    freq = df.index.freq
    if tmax is tmin is dt is None:
        dt = 1
    if tmin is None:
        tmin = df.index.min()
    if tmax is None:
        tmax = df.index.max()
    if dt is not None:
        if isinstance(dt, int):
            if dt > 0:
                tmax += dt * freq
            elif dt < 0:
                tmin += dt * freq
        else:
            dt = pd.to_timedelta(dt)
            if dt > pd.to_timedelta('0d'):
                tmax += dt
            else:
                tmin -= dt
    index = pd.date_range(tmin, tmax, freq=freq)
    return df.reindex(index)


def suptitle(fig, text=None, **kw):
    """Add a nice left-aligned suptitle."""
    if text is None:
        fig, text = plt.gcf(), fig
    fig = fig.figure or fig
    fig.text(fig.subplotpars.left, .99, text, ha='left', va='top', size='large', **kw)
    
    
def rlabel(ax, label=None, **kw):
    """Add a right-side axis title."""
    if label is None:
        ax, label = plt.gca(), ax
    bbox = kw.pop('bbox', dict(facecolor='.9', alpha=0.2))
    ax.text(1, .5, label,
            rotation=-90, ha='left', va='center', transform=ax.transAxes,
            bbox=bbox, **kw)
    

def xdate(ax, fmt=None, freq=None):
    """Tweak x-axis date formatting."""
    dates = plt.matplotlib.dates
    if fmt is None:
        ax, fmt = plt.gca(), ax
    if freq:
        t1, t2 = dates.num2date(ax.get_xticks()[[0,-1]])
        ticks = pd.date_range(t1, t2, freq=freq)
        ax.set_xticks(ticks)
    ax.xaxis.set_major_formatter(dates.DateFormatter(fmt))
    
    
def plot_tsresiduals(Y, y, acf_lags=np.r_[1:26]):
    """Plot timeseries residuals for ground truth Y and estimate y."""
    fig = plt.figure()
    gs = plt.GridSpec(3, 2, figure=fig)
    ts_ax = fig.add_subplot(gs[0,:])
    axs = np.array([ts_ax] + [fig.add_subplot(gs[i,j]) for j in (0,1) for i in (1,2)])
    ax, rax, hax, acfax, pacfax = axs
    #((ax, hax), (rax, acfax)) = axs
    mask = ~(np.isnan(Y) | np.isnan(y))
    Y, y = Y[mask], y[mask]
    #dy = y - Y
    # I was surprised by this convention but ok
    dy = Y - y
    ax.plot(Y, color='k')
    ax.plot(y)
    ax.set(title='Time Series')
    lim = 1.1 * max(-dy.min(), dy.max())
    lim = -lim, lim
    rax.plot(dy)
    rax.set(ylim=lim, title='Residuals')
    sns.distplot(dy, bins=np.linspace(lim[0], lim[1], 22),
                 hist=True, kde=True, rug=True, ax=hax)
    hax.set(title='Residual Distribution')
    sm.graphics.tsa.plot_acf(dy, lags=acf_lags, ax=acfax)
    sm.graphics.tsa.plot_pacf(dy, lags=acf_lags, ax=pacfax)
    for a in axs.ravel():
        a.grid()
    plt.tight_layout()
    return fig, axs


def RMSE(Y, y):
    """Root-mean-square error."""
    return np.sqrt(np.mean((Y-y)**2))
def MAE(Y, y):
    """Mean absolute error."""
    return np.mean(np.abs(Y-y))
def MAPE(Y, y):
    """Mean absolute percent error."""
    return 100 * np.mean(np.abs((Y-y)/Y))
def MASE(Y, y):
    """TODO"""
    return np.nan # TODO


def tsaccuracy(Ytest, models):
    """Gather some metrics for a few models."""
    fs = RMSE, MAE, MAPE, MASE
    return pd.DataFrame({
        label: [ f(Ytest, model.predict(Ytest.index.min(), Ytest.index.max()))
                for f in (RMSE, MAE, MAPE, MASE) ]
        for (label, model) in models.items()
    }, index=[f.__name__ for f in fs]).T

def ciclean(ci_df):
    """Clean up conf_int() result column names."""
    ci_df = ci_df.copy()
    ci_df.columns = 'lower', 'upper'
    return ci_df


legend_right = dict(loc='center left', bbox_to_anchor=[1, .5])

In [4]:
!apt install pmdarima as pmd

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package pmdarima
E: Unable to locate package as
E: Unable to locate package pmd


In [5]:
!pip install rdatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rdatasets: filename=rdatasets-0.1.0-py3-none-any.whl size=19168565 sha256=f0bde3e7219802d52dc6bd842fe8ddf590c1047dfe96e8103545503319400b06
  Stored in directory: /root/.cache/pip/wheels/a2/7b/d8/1e6f1b7550e3441a89c5a165ab89a1052e58b99bf58a2c6dd5
Successfully built rdatasets


In [6]:
import sys
!pip install utils
sys.path.insert(0, '.')
from utils import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://robjhyndman.com/hyndsight/ts2csv/

In [7]:
from rdatasets import summary
summary()

,Package,Item,Title,Rows,Cols,n_binary,n_character,n_factor,n_logical,n_numeric,CSV,Doc
0,boot,acme,Monthly Excess Returns,60,3,0,1,0,0,2,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
1,boot,aids,Delay in AIDS Reporting in England and Wales,570,6,1,0,0,0,6,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
2,boot,aircondit,Failures of Air-conditioning Equipment,12,1,0,0,0,0,1,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
3,boot,aircondit7,Failures of Air-conditioning Equipment,24,1,0,0,0,0,1,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
4,boot,amis,Car Speeding and Warning Signs,8437,4,1,0,0,0,4,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
...,...,...,...,...,...,...,...,...,...,...,...,...
1340,Zelig,tobin,Tobin's Tobit Data,20,3,0,0,0,0,3,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
1341,Zelig,turnout,Turnout Data Set from the National Election Su...,2000,5,2,0,1,0,4,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
1342,Zelig,voteincome,Sample Turnout and Demographic Data from the 2...,1500,7,3,0,1,0,6,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...
1343,Zelig,Weimar,1932 Weimar election data,10,11,0,0,0,0,11,https://raw.github.com/vincentarelbundock/Rdat...,https://raw.github.com/vincentarelbundock/Rdat...


In [8]:
from rdatasets import data
dataset = data('PlantGrowth')
dataset

,weight,group
0,4.17,ctrl
1,5.58,ctrl
2,5.18,ctrl
3,6.11,ctrl
4,4.50,ctrl
5,4.61,ctrl
6,5.17,ctrl
7,4.53,ctrl
8,5.33,ctrl
9,5.14,ctrl


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import sys
sys.path.insert(0, '.')
# from utils import *

# extra import for feature engineering
!pip install tsfresh
import tsfresh

# also sklearn for a little PCA + t-SNE
from sklearn import preprocessing, decomposition, manifold

Importing Data

In [ ]:

path = 'https://raw.githubusercontent.com/pawarbi/datasets/master/timeseries/ts_frenchretail.csv'

data = pd.read_csv(path, parse_dates=True, index_col="Date")

data.index.freq='Q'

data.head()

In [ ]:
path = 'https://robjhyndman.com/hyndsight/ts2csv'

data = pd.read_csv(path, parse_dates=True)


data.head()